In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline

#Machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

Populating the interactive namespace from numpy and matplotlib


В файле размещены данные обо всех сессиях всех игроков игры X за последние 90 дней 
(udid - id игрока, time - дата, count - количество сессий в день, avg_duration - средняя продолжительность сессии 
в секундах). Постройте на основании этих данных модель прогнозирования ухода игрока. Применяемые методы и 
инструменты ограничены лишь вашей фантазией и умениями. Главное, что модель в итоге должна уметь прогнозировать 
для каждого конкретного игрока с какой-либо вероятностью то, что он в ближайшее время покинет проект. 
Покинувшим проект игроком давайте считать того, кто не заходил в игру уже минимум две недели. 

In [2]:
df = pd.read_csv('TestGameSessions.csv')
df.head()

,udid,time,count,avg_duration
0,104008439,1474588800,2,112
1,104011104,1474588800,2,44
2,104022241,1474588800,2,413
3,104022356,1474588800,2,125
4,104022069,1474592400,5,1561


Приведем время к формату datetime

In [3]:
df['time']= pd.to_datetime(df['time'],unit='s')
df.head()

,udid,time,count,avg_duration
0,104008439,2016-09-23 00:00:00,2,112
1,104011104,2016-09-23 00:00:00,2,44
2,104022241,2016-09-23 00:00:00,2,413
3,104022356,2016-09-23 00:00:00,2,125
4,104022069,2016-09-23 01:00:00,5,1561


Оставим только дату и отсортируем по возрастанию

In [4]:
df['time'] = df['time'].dt.date

In [5]:
df.sort_values(by='time', ascending=True)

,udid,time,count,avg_duration
0,104008439,2016-09-23,2,112
80,103997007,2016-09-23,1,123
79,104022409,2016-09-23,1,11396
78,104022106,2016-09-23,1,1928
77,104021079,2016-09-23,3,37
76,103996866,2016-09-23,8,2820
75,104022298,2016-09-23,3,3320
74,104019815,2016-09-23,4,1657
73,104012689,2016-09-23,3,935
72,103968104,2016-09-23,2,26


Последнее число в списке-21.12.2016, значит ушедшими считаются те, кто не заходил после 07.12.2016. Сгруппируем данные по уникальным id игроков и создадим новый столбик churn (1-ушел, 0-остался)

In [6]:
df = df.groupby('udid').agg({'count':'sum', 'avg_duration': 'mean','time': 'max' })

In [7]:
churn =[]
for time in df['time']:
    if str(time) > '2016-12-07':
        churn.append(0)
    else:
        churn.append(1)
df['churn'] = churn

In [8]:
df.head()

,count,avg_duration,time,churn
udid,,,,
74385162,1,39.0,2016-10-31,1
79599601,2,892.0,2016-11-15,1
91475825,4,5.5,2016-10-11,1
91492531,1,79.0,2016-10-08,1
92137561,1,9.0,2016-12-12,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4267 entries, 74385162 to 104031228
Data columns (total 4 columns):
count           4267 non-null int64
avg_duration    4267 non-null float64
time            4267 non-null object
churn           4267 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 166.7+ KB


In [10]:
df.churn.value_counts()

1    3523
0     744
Name: churn, dtype: int64

Для построения моделей столбец time нам не нужен, поэтому удаляем его

In [11]:
new_df = df.drop(['time'], axis=1)

In [12]:
new_df.head()

,count,avg_duration,churn
udid,,,
74385162,1,39.0,1
79599601,2,892.0,1
91475825,4,5.5,1
91492531,1,79.0,1
92137561,1,9.0,0


In [13]:
new_df = new_df.reset_index()

Разделяем датасет на тестовую и тренировочную части

In [14]:
train, test = train_test_split(new_df, test_size=0.2)

Строим модели по двум признакам - количесто и средняя длина сессий, остальные столбцы можно удалить

In [15]:
X_train = train.drop(['udid','churn'], axis=1) 
Y_train = train['churn']
X_test = test.drop(['udid','churn'], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((3413, 2), (3413,), (854, 2))

Для прогнозирования я использовала следующие модели: 

In [16]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train)*100, 2)
acc_log

82.89

In [17]:
#k-Nearest Neighbors algorithm

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train)*100, 2)
acc_knn

85.76

In [18]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train)*100, 2)
acc_gaussian

82.42

In [19]:
# Random Forest

random_forest = RandomForestClassifier()
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train)*100, 2)
acc_random_forest

93.5

In [20]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train)*100, 2)
acc_decision_tree

95.25

Сводные данные по всем моделям:

In [21]:
models = pd.DataFrame({'Model':['Logistic regression', 'KNN', 'Naive Bayes', 'Random forest', 'Decision tree' ],
                      'Score':[acc_log, acc_knn, acc_gaussian, acc_random_forest, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
4,Decision tree,95.25
3,Random forest,93.50
1,KNN,85.76
0,Logistic regression,82.89
2,Naive Bayes,82.42


Наибольшая точность у модели дерева решений

In [22]:
submission = pd.DataFrame({'udid':test['udid'], 'churn':Y_pred})
submission.churn.value_counts()

1    712
0    142
Name: churn, dtype: int64